In [1]:
!wget https://mlbootcamp.ru/media/condition/mlboot_dataset.zip

--2018-07-14 17:23:21--  https://mlbootcamp.ru/media/condition/mlboot_dataset.zip
Resolving mlbootcamp.ru (mlbootcamp.ru)... 185.5.138.249
Connecting to mlbootcamp.ru (mlbootcamp.ru)|185.5.138.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2738062403 (2.5G) [application/zip]
Saving to: 'mlboot_dataset.zip'

mlboot_dataset.zip  100%[===================>]   2.55G  59.7MB/s    in 45s     

2018-07-14 17:24:06 (58.4 MB/s) - 'mlboot_dataset.zip' saved [2738062403/2738062403]



In [3]:
!unzip mlboot_dataset.zip

Archive:  mlboot_dataset.zip
   creating: mlboot_dataset/
  inflating: mlboot_dataset/mlboot_test.tsv.gz  
   creating: __MACOSX/
   creating: __MACOSX/mlboot_dataset/
  inflating: __MACOSX/mlboot_dataset/._mlboot_test.tsv.gz  
  inflating: mlboot_dataset/.DS_Store  
  inflating: __MACOSX/mlboot_dataset/._.DS_Store  
  inflating: mlboot_dataset/mlboot_data.tsv.gz  
  inflating: __MACOSX/mlboot_dataset/._mlboot_data.tsv.gz  
  inflating: mlboot_dataset/mlboot_train_answers.tsv.gz  
  inflating: __MACOSX/mlboot_dataset/._mlboot_train_answers.tsv.gz  
  inflating: __MACOSX/._mlboot_dataset  


In [1]:
!ls -la

total 2673964
drwx------ 11 root root       4096 Jul 14 18:11 .
drwxr-xr-x 23 root root       4096 Jul 14 17:12 ..
-rw-------  1 root root        487 Jul 14 18:00 .bash_history
-rw-r--r--  1 root root       3106 Apr  9 11:10 .bashrc
drwx------  4 root root       4096 Jul 14 17:34 .cache
-rw-r--r--  1 root root          0 Jul 14 17:12 .cloud-locale-test.skip
drwx------  3 root root       4096 Jul 14 17:34 .config
drwxr-xr-x  2 root root       4096 Jul 14 17:23 .ipynb_checkpoints
drwxr-xr-x  5 root root       4096 Jul 14 17:22 .ipython
drwxr-xr-x  2 root root       4096 Jul 14 18:05 .jupyter
drwx------  3 root root       4096 Jul 14 17:19 .local
-rw-r--r--  1 root root        148 Aug 17  2015 .profile
drwx------  2 root root       4096 Jul 14 17:12 .ssh
drwxrwxr-x  3 root root       4096 Jun 25 13:57 __MACOSX
-rw-r--r--  1 root root      10164 Jul 14 18:11 mlboot.ipynb
drwx------  2 root root       4096 Jun 25 13:56 mlboot_dataset
-rw-r--r--  1 root root 2738062403 Jun 25 14:10 mlboot_da

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import json

from datetime import datetime

import lightgbm as lgb
import gc

In [2]:
answers = pd.read_table('mlboot_dataset/mlboot_train_answers.tsv.gz')
testdf = pd.read_table('mlboot_dataset/mlboot_test.tsv.gz')

In [4]:
chunksize = 19528597 // 1 + 1
chunksize

19528598

In [6]:
%%time

iter_counter = 0

result = pd.DataFrame()

print(datetime.now(), 'start reading')

for df in pd.read_table('mlboot_dataset/mlboot_data.tsv.gz', header=None, chunksize=chunksize):
    
    print(datetime.now(), 'read')
    
    iter_counter = iter_counter + 1    
    print(datetime.now(), ' Iteration: ', iter_counter)
    
    df = df.merge(df.groupby(0)[0].count().reset_index(name = 'tx'), how = 'left')
    
    df['var'] = df.groupby(0)[5].transform('var')
    
    print(datetime.now(), 'entering train/test')
    
    dfm_train = df.merge(answers, left_on=0, right_on='cuid', how='inner')
    dfm_test = df.merge(testdf, left_on=0, right_on='cuid', how='inner')
    
    dfm_train_len = dfm_train.shape[0]
    dfm2 = dfm_train.append(dfm_test)
    dfm_train_len, dfm2.shape
    
    print(datetime.now(), 'train/test created')
    
    dfm2['semicolon2'] = dfm2[2].fillna('').apply(lambda x: x.count(":"))
    dfm2['ones2'] = dfm2[2].fillna('').apply(lambda x: x.count(":1"))
    dfm2['twos2'] = dfm2[2].fillna('').apply(lambda x: x.count(":2"))
    
    dfm2['semicolon3'] = dfm2[3].fillna('').apply(lambda x: x.count(":"))
    dfm2['ones3'] = dfm2[3].fillna('').apply(lambda x: x.count(":1"))
    dfm2['twos3'] = dfm2[3].fillna('').apply(lambda x: x.count(":2"))
    
    dfm2['semicolon4'] = dfm2[4].fillna('').apply(lambda x: x.count(":"))
    dfm2['ones4'] = dfm2[4].fillna('').apply(lambda x: x.count(":1"))
    dfm2['twos4'] = dfm2[4].fillna('').apply(lambda x: x.count(":2"))
    
    print(datetime.now(), 'entering tfidf')
        
    dfm2[2].fillna('{}', inplace=True)
    dfm2_2 = TfidfVectorizer(ngram_range=(1, 2), max_features=50_000).fit_transform(dfm2[2].fillna(''))
    
    print(datetime.now(), '2 tfidf done')
    
    dfm2[3].fillna('{}', inplace=True)
    dfm2_3 = TfidfVectorizer(ngram_range=(1, 2), max_features=50_000).fit_transform(dfm2[3].fillna(''))
    
    print(datetime.now(), '3 tfidf done')
    
    dfm2[4].fillna('{}', inplace=True)
    dfm2_4 = TfidfVectorizer(ngram_range=(1, 2), max_features=50_000).fit_transform(dfm2[4].fillna(''))
    
    print(datetime.now(), 'hstack')
    
    dfms = scipy.sparse.hstack([dfm2['tx'].values.reshape(-1,1), dfm2['var'].values.reshape(-1,1), dfm2['semicolon2'].values.reshape(-1,1), dfm2['ones2'].values.reshape(-1,1), dfm2['twos2'].values.reshape(-1,1), dfm2['semicolon3'].values.reshape(-1,1), dfm2['ones3'].values.reshape(-1,1), dfm2['twos3'].values.reshape(-1,1), dfm2['semicolon4'].values.reshape(-1,1), dfm2['ones4'].values.reshape(-1,1), dfm2['twos4'].values.reshape(-1,1),dfm2_2, dfm2_3, dfm2_4, dfm2[1].values.reshape(-1,1), dfm2[5].values.reshape(-1,1)]).tocsr()

    y_train = dfm2['target'][:dfm_train_len].astype(int)
    X_train = dfms[:dfm_train_len]
    X_test = dfms[dfm_train_len:]
    
    res = pd.DataFrame()
    
    foldno = 0
    
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=10, shuffle=False)
    
    for train_index, test_index in kf.split(X_train):
        print("TRAIN:", train_index, "\nTEST:", test_index)
        X_tr, X_va = X_train[train_index], X_train[test_index]
        y_tr, y_va = y_train[train_index], y_train[test_index]
    
        #X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=False)
        X_te = X_test
        #X_tr.shape, X_te.shape, X_va.shape
        
        foldno = foldno + 1

        print(datetime.now(), ' FOLD ', foldno)


        tr_data = lgb.Dataset(X_tr, label=y_tr)
        va_data = lgb.Dataset(X_va, label=y_va, reference=tr_data)

        gc.collect()

        parameters = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc',
            #'num_leaves': 17,
            #'learning_rate': 0.05,
            'feature_fraction': 0.5,
            #'bagging_fraction': 0.9,
            'bagging_freq': 4,
            'verbose': 10
        }

        model = lgb.train(parameters,
                          tr_data,
                          valid_sets=va_data,
                          num_boost_round=2500,
                          early_stopping_rounds=50,
                          verbose_eval=5)

        res['cuid'] = dfm2[0][dfm_train_len:]
        res['target'] = model.predict(X_test)

        result = result.append(res)
        
        result.to_csv('result_'+str(foldno)+'.csv')

2018-07-22 11:18:53.292670 start reading
2018-07-22 11:22:14.419286 read
2018-07-22 11:22:14.419466  Iteration:  1
2018-07-22 11:22:36.646016 entering train/test


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


2018-07-22 11:22:59.677548 train/test created
2018-07-22 11:26:10.550100 entering tfidf
2018-07-22 11:43:21.597805 2 tfidf done
2018-07-22 12:47:20.073626 3 tfidf done
2018-07-22 12:49:33.004803 hstack
TRAIN: [ 1287435  1287436  1287437 ... 12874342 12874343 12874344] 
TEST: [      0       1       2 ... 1287432 1287433 1287434]
2018-07-22 12:52:22.289023  FOLD  1
Training until validation scores don't improve for 50 rounds.
[5]	valid_0's auc: 0.566326
[10]	valid_0's auc: 0.574065
[15]	valid_0's auc: 0.577702
[20]	valid_0's auc: 0.580134
[25]	valid_0's auc: 0.582619
[30]	valid_0's auc: 0.583602
[35]	valid_0's auc: 0.58529
[40]	valid_0's auc: 0.585909
[45]	valid_0's auc: 0.587424
[50]	valid_0's auc: 0.58871
[55]	valid_0's auc: 0.589444
[60]	valid_0's auc: 0.590201
[65]	valid_0's auc: 0.590686
[70]	valid_0's auc: 0.590623
[75]	valid_0's auc: 0.590928
[80]	valid_0's auc: 0.591338
[85]	valid_0's auc: 0.591784
[90]	valid_0's auc: 0.592109
[95]	valid_0's auc: 0.59238
[100]	valid_0's auc: 0.59

[25]	valid_0's auc: 0.597857
[30]	valid_0's auc: 0.599248
[35]	valid_0's auc: 0.599379
[40]	valid_0's auc: 0.600077
[45]	valid_0's auc: 0.60102
[50]	valid_0's auc: 0.601517
[55]	valid_0's auc: 0.602162
[60]	valid_0's auc: 0.602562
[65]	valid_0's auc: 0.602734
[70]	valid_0's auc: 0.603221
[75]	valid_0's auc: 0.603468
[80]	valid_0's auc: 0.603519
[85]	valid_0's auc: 0.603816
[90]	valid_0's auc: 0.603968
[95]	valid_0's auc: 0.6043
[100]	valid_0's auc: 0.604501
[105]	valid_0's auc: 0.604554
[110]	valid_0's auc: 0.604912
[115]	valid_0's auc: 0.604865
[120]	valid_0's auc: 0.605034
[125]	valid_0's auc: 0.60556
[130]	valid_0's auc: 0.60564
[135]	valid_0's auc: 0.605725
[140]	valid_0's auc: 0.605646
[145]	valid_0's auc: 0.605876
[150]	valid_0's auc: 0.605949
[155]	valid_0's auc: 0.606056
[160]	valid_0's auc: 0.606062
[165]	valid_0's auc: 0.606189
[170]	valid_0's auc: 0.606154
[175]	valid_0's auc: 0.606274
[180]	valid_0's auc: 0.606435
[185]	valid_0's auc: 0.606137
[190]	valid_0's auc: 0.60612
[

[75]	valid_0's auc: 0.602914
[80]	valid_0's auc: 0.603154
[85]	valid_0's auc: 0.603387
[90]	valid_0's auc: 0.60372
[95]	valid_0's auc: 0.604032
[100]	valid_0's auc: 0.604384
[105]	valid_0's auc: 0.604501
[110]	valid_0's auc: 0.604532
[115]	valid_0's auc: 0.604555
[120]	valid_0's auc: 0.604624
[125]	valid_0's auc: 0.604701
[130]	valid_0's auc: 0.60482
[135]	valid_0's auc: 0.605008
[140]	valid_0's auc: 0.605033
[145]	valid_0's auc: 0.605146
[150]	valid_0's auc: 0.605099
[155]	valid_0's auc: 0.605065
[160]	valid_0's auc: 0.605245
[165]	valid_0's auc: 0.605448
[170]	valid_0's auc: 0.605654
[175]	valid_0's auc: 0.605603
[180]	valid_0's auc: 0.605788
[185]	valid_0's auc: 0.605735
[190]	valid_0's auc: 0.605699
[195]	valid_0's auc: 0.605526
[200]	valid_0's auc: 0.605385
[205]	valid_0's auc: 0.605536
[210]	valid_0's auc: 0.605717
[215]	valid_0's auc: 0.605738
[220]	valid_0's auc: 0.605685
[225]	valid_0's auc: 0.60599
[230]	valid_0's auc: 0.605994
[235]	valid_0's auc: 0.606002
[240]	valid_0's au

[295]	valid_0's auc: 0.622522
[300]	valid_0's auc: 0.622614
Early stopping, best iteration is:
[250]	valid_0's auc: 0.622815
CPU times: user 4d 18h 57min 29s, sys: 17min 58s, total: 4d 19h 15min 28s
Wall time: 8h 26min


In [7]:
resultdf = result.groupby('cuid').mean()
        
resultdf['cuid'] = resultdf.index
testdf = testdf.merge(resultdf, left_on='cuid', right_on='cuid', how='outer')
testdf.sample(4)

testdf['target'].to_csv('kfold10.tsv', header=None, index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: 'cuid' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.
